In [1]:
# Alright, let's start with some easy stuff. Get the top volume of a stock in yesterday, 3 days, 5 days , 15 days and 30 days.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool


In [2]:
def get_top_volume(file):
    # Get symbol
    symbol = os.path.splitext(file)[0]
    df = pd.read_csv(os.path.join(Data_Path, file))
    # The dataframe should be sorted by datetime already, but just to be sure, sort it again.
    df = df.sort_values(by='datetime', ascending=False)
    # Get volume of yesterday
    yesterday_volume = df.iloc[0]['volume'].mean()
    # Get volume average of 3 days
    volume_3_days = df.iloc[0:3]['volume'].mean()
    # Get volume of 5 days
    volume_5_days = df.iloc[0:5]['volume'].mean()
    # Get volume of 15 days
    volume_15_days = df.iloc[0:15]['volume'].mean()
    # Get volume of 30 days
    volume_30_days = df.iloc[0:30]['volume'].mean()
    # Append to result
    data = {
        'symbol': symbol,
        'yesterday_volume': yesterday_volume,
        'volume_3_days': volume_3_days,
        'volume_5_days': volume_5_days,
        'volume_15_days': volume_15_days,
        'volume_30_days': volume_30_days
    }
    return data

In [3]:
# Idea is simple. Read in each file, caculate volume of yesterday, 3 days, 5 days, 15 days and 30 days.
# This should be easy to do with a multiprocessing, but I will use tranditional approach for now.
# For references, the production data's csv has the following columns:
# open,high,low,close,volume,timestamp,datetime

# This is Mac data path
#Data_Path = '/Users/Arthur/Developer/Trading/Data/Daily/Production'
# This is WSL2 data path
Data_Path = '/home/arthur/Developer/Trading/Data/Daily/Production'
Working_Path = os.getcwd()
# Get file list
file_list = [f for f in os.listdir(Data_Path) if f.endswith('.csv')]
# Get symbol list
symbol_list = [os.path.splitext(f)[0] for f in file_list]
pd.options.display.float_format = '{:,.2f}'.format
# file_list = ['AAPL.csv', 'SPY.csv', 'GOOG.csv', 'MSFT.csv']

top_volume_list =pd.DataFrame()
# with Pool(processes = os.cpu_count()) as pool:
#     for result in tqdm(pool.imap_unordered(get_top_volume, file_list), total=len(file_list)):
#         top_volume_list = top_volume_list.append(result)
pool = Pool(processes=os.cpu_count())
result = [f for f in tqdm(pool.imap_unordered(get_top_volume, file_list), total=len(file_list))]
pool.close()
pool.join()

top_volume_list = pd.DataFrame(result)
top_volume_list.to_csv(os.path.join(Working_Path,'top_volume_list.csv'), index=False)

100%|██████████| 12285/12285 [00:08<00:00, 1488.85it/s]


In [4]:
top_volume_list.sort_values(by='yesterday_volume', ascending=False, inplace=True)

In [5]:
top_volume_list.head(50)

,symbol,yesterday_volume,volume_3_days,volume_5_days,volume_15_days,volume_30_days
905,SQQQ,"175,486,690.00","226,450,710.00","216,061,004.00","164,090,195.07","141,813,534.83"
8186,AAPL,"107,496,982.00","120,753,089.33","137,746,065.60","106,655,399.20","86,615,035.00"
6567,LCID,"103,929,805.00","85,846,501.33","85,750,082.60","109,645,393.93","107,897,845.40"
11224,SPY,"98,977,532.00","121,315,645.67","128,998,474.00","86,893,040.00","70,903,742.17"
6120,SNDL,"90,743,158.00","105,116,204.00","115,108,542.40","133,072,451.87","138,326,554.57"
7757,F,"88,147,408.00","100,854,947.67","106,256,316.20","90,926,335.40","108,505,553.20"
11465,ISIG,"84,523,423.00","28,190,311.33","16,921,668.60","5,664,154.13","2,893,196.00"
10698,CCL,"78,131,555.00","71,402,480.33","71,404,289.00","50,312,482.13","39,027,753.30"
7160,CFVI,"77,075,622.00","74,332,498.00","44,615,844.00","14,909,207.60","7,475,667.87"
9782,TQQQ,"76,206,983.00","98,486,218.67","89,426,791.20","56,498,919.13","46,168,195.50"
